In [1]:
import moxing as mox
mox.file.copy_parallel(src_url="s3://focus/nlp/data/", dst_url='../data/')
mox.file.copy_parallel(src_url="s3://focus/nlp/nlp_vqa/", dst_url='.')

INFO:root:Using MoXing-v2.0.0.rc2.4b57a67b-4b57a67b
INFO:root:Using OBS-Python-SDK-3.20.9.1
INFO:root:Listing OBS: 1000
INFO:root:Listing OBS: 2000
INFO:root:Listing OBS: 3000
INFO:root:Listing OBS: 4000
INFO:root:Listing OBS: 5000
INFO:root:Listing OBS: 6000
INFO:root:Listing OBS: 7000
INFO:root:Listing OBS: 8000
INFO:root:Listing OBS: 9000
INFO:root:Listing OBS: 10000
INFO:root:Listing OBS: 11000
INFO:root:Listing OBS: 12000
INFO:root:Listing OBS: 13000
INFO:root:Listing OBS: 14000
INFO:root:Listing OBS: 15000
INFO:root:Listing OBS: 16000
INFO:root:Listing OBS: 17000
INFO:root:Listing OBS: 18000
INFO:root:Listing OBS: 19000
INFO:root:Listing OBS: 20000
INFO:root:Listing OBS: 21000
INFO:root:Listing OBS: 22000
INFO:root:Listing OBS: 23000
INFO:root:Listing OBS: 24000
INFO:root:Listing OBS: 25000
INFO:root:Listing OBS: 26000
INFO:root:Listing OBS: 27000
INFO:root:Listing OBS: 28000
INFO:root:Listing OBS: 29000
INFO:root:Listing OBS: 30000
INFO:root:Listing OBS: 31000
INFO:root:Listing 

In [1]:
!pip install tqdm

Looking in indexes: http://repo.myhuaweicloud.com/repository/pypi/simple


In [ ]:
import moxing as mox
mox.file.copy_parallel(src_url="s3://focus/nlp/nlp_vqa/san.py", dst_url='./san.py')
mox.file.copy_parallel(src_url="s3://focus/nlp/nlp_vqa/dataset.py", dst_url='./dataset.py')

In [ ]:
import sys
import os.path
import mindspore
from mindspore import Tensor, nn, Model, context
from mindspore import load_checkpoint, load_param_into_net
from mindspore import ops
from mindspore.ops import functional as F
from mindspore.ops import composite as C
from mindspore.common.parameter import ParameterTuple
from mindspore.train.callback import LossMonitor, CheckpointConfig, ModelCheckpoint, TimeMonitor
import numpy as np
from tqdm import tqdm
import config
import dataset
import san
import utils
import mindspore.context as context
import json
import math
from datetime import datetime
from bert4ms import models 
from mindspore.nn import TrainOneStepCell
context.set_context(mode=context.PYNATIVE_MODE, device_target='Ascend')

In [ ]:
class WithLossCell(nn.Cell):
    def __init__(self, model):
        super(WithLossCell, self).__init__(auto_prefix=False)
        self.loss = nn.SoftmaxCrossEntropyWithLogits()
        self.net = model

    def construct(self, q, a, img):
        out = self.net(q, img)
        loss = self.loss(out, a)
        batch_loss = loss.asnumpy().mean()
        return batch_loss
    

class WithAccuracy(nn.Cell):
    def __init__(self, model):
        super(WithAccuracy, self).__init__(auto_prefix=False)
        self.net = model

    def construct(self, q, a, img):
        out = self.net(q, img)
        out = ops.Argmax(output_type=mindspore.int32)(out)
        predicted = out.asnumpy()
        ans = a.asnumpy()
        batch_size = ans.shape[0]
        acc = 0
        for i in range(batch_size):
            if ans[i,predicted[i]]!=0:
                acc += 1
        accuracy = acc / batch_size
        return accuracy, predicted

In [4]:
if __name__ == '__main__':

    val_loader = dataset.get_loader(val=True)

    model = san.SANModel()
    opt = nn.Adam(params=model.trainable_params())
    net_with_loss = WithLossCell(model)
    train_net = TrainOneStepCell(net_with_loss, opt)
    train_net.set_train(True)
    
    eval_net = WithAccuracy(model)
    eval_net.set_train(False)

    for epoch in range(config.epochs):
        losses = []
        accs = []
        train_loader = dataset.get_loader(train=True)
        
        step = 0
        tq = tqdm(train_loader, desc='{} EPOCH{:02d}'.format('train', epoch), ncols=0, total=math.ceil(len(train_loader.source.questions) / config.batch_size))
        for q, a, img in tq:
            loss = train_net(q, a, img)
            print("Epoch:{},Step:{},Loss:{}".format(epoch,step,loss))
            losses.append(loss)
        print("Epoch {} done.Train Loss:{}".format(epoch,np.array(losses).mean()))
        
        step = 0
        tq = tqdm(val_loader, desc='{} EPOCH{:02d}'.format('val', epoch), ncols=0, total=math.ceil(len(val_loader.source.questions) / config.batch_size))
        for q, a, img in tq:
            acc,predicted = eval_net(q, a, img)
            print("Epoch:{},Step:{},Accuracy:{},Predicted:{}".format(epoch,step,acc,predicted))
            accs.append(acc)
        print("Epoch {} done.Val Accuracy:{}".format(epoch,np.array(accs).mean()))

INFO:root:The checkpoint file not found, start to download.


Generating answers vocab...
Answers vocab is generated


INFO:root:The checkpoint file not found, start to download.
INFO:root:The checkpoint file not found, start to download.
[WARNING] ME(36314:281473397931616,MainProcess):2022-06-26-17:23:16.888.416 [mindspore/train/serialization.py:581] Remove parameter prefix name: bert., continue to load.
INFO:root:The checkpoint file not found, start to download.


Generating answers vocab...
Answers vocab is generated


train EPOCH00:   0% 1/5768 [03:46<362:33:59, 226.33s/it]

...

Epoch:0,Step:0,Loss:[224.44504    11.204501    6.853778    4.6784163]


train EPOCH00:   1% 72/5768 [07:16<4:38:41,  2.94s/it]

Epoch:0,Step:0,Loss:[232.34811   229.8594      4.5846653  29.568674 ]


train EPOCH00:   1% 72/5768 [07:19<9:39:31,  6.10s/it]


KeyboardInterrupt: 